In [1]:
import ast
import json
import warnings

import pandas as pd
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.metrics import accuracy_score
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.preprocessing import OneHotEncoder
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.registry import Registry
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
from snowflake.snowpark import types as T
from snowflake.snowpark.functions import col

warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
session = Session.builder.configs(SnowflakeLoginOptions()).getOrCreate()

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


In [3]:
titanic_df = session.table("titanic")

In [4]:
titanic_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SURVIVED"  |"PCLASS"  |"AGE"  |"SIBSP"  |"PARCH"  |"FARE"   |"ADULT_MALE"  |"DECK"  |"ALIVE"  |"ALONE"  |"SEX"   |"EMBARKED"  |"CLASS"  |"WHO"  |"EMBARK_TOWN"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0           |3         |22.00  |1        |0        |7.2500   |True          |NULL    |False    |False    |MALE    |S           |THIRD    |MAN    |SOUTHAMPTON    |
|1           |1         |38.00  |1        |0        |71.2833  |False         |C       |True     |False    |FEMALE  |C           |FIRST    |WOMAN  |CHERBOURG      |
|1           |3         |26.00  |0        |0        |7.9250   |False         |NULL    |True     |True     |FEMALE  |S           |THIRD    |WOMAN  |SOUTHAMPTON    |
|1           |1 

In [5]:
# Columns with null values and their respective counts
{
    k: v
    for k, v in {
        col_name: titanic_df.where(col(col_name).is_null()).count()
        for col_name in titanic_df.columns
    }.items()
    if v > 0
}

{'AGE': 177, 'DECK': 688, 'EMBARKED': 2, 'EMBARK_TOWN': 2}

In [6]:
titanic_df = titanic_df.drop(
    ["AGE", "DECK", "ALIVE", "ADULT_MALE", "EMBARKED", "SEX", "PCLASS", "ALONE"]
)

In [7]:
titanic_df = titanic_df.withColumn("FARE", titanic_df["FARE"].astype(T.FloatType()))

titanic_df.show()

------------------------------------------------------------------------------
|"SURVIVED"  |"SIBSP"  |"PARCH"  |"CLASS"  |"WHO"  |"EMBARK_TOWN"  |"FARE"   |
------------------------------------------------------------------------------
|0           |1        |0        |THIRD    |MAN    |SOUTHAMPTON    |7.25     |
|1           |1        |0        |FIRST    |WOMAN  |CHERBOURG      |71.2833  |
|1           |0        |0        |THIRD    |WOMAN  |SOUTHAMPTON    |7.925    |
|1           |1        |0        |FIRST    |WOMAN  |SOUTHAMPTON    |53.1     |
|0           |0        |0        |THIRD    |MAN    |SOUTHAMPTON    |8.05     |
|0           |0        |0        |THIRD    |MAN    |QUEENSTOWN     |8.4583   |
|0           |0        |0        |FIRST    |MAN    |SOUTHAMPTON    |51.8625  |
|0           |3        |1        |THIRD    |CHILD  |SOUTHAMPTON    |21.075   |
|1           |0        |2        |THIRD    |WOMAN  |SOUTHAMPTON    |11.1333  |
|1           |1        |0        |SECOND   |CHILD  |

In [8]:
cat_cols = ["CLASS", "WHO", "EMBARK_TOWN"]
num_cols = ["SIBSP", "PARCH", "FARE"]

In [9]:
impute_cat = SimpleImputer(
    input_cols=cat_cols,
    output_cols=cat_cols,
    strategy="most_frequent",
    drop_input_cols=True,
)

titanic_df = impute_cat.fit(titanic_df).transform(titanic_df)
titanic_df.show()

------------------------------------------------------------------------------
|"CLASS"  |"WHO"  |"EMBARK_TOWN"  |"SURVIVED"  |"SIBSP"  |"PARCH"  |"FARE"   |
------------------------------------------------------------------------------
|THIRD    |MAN    |SOUTHAMPTON    |0           |1        |0        |7.25     |
|FIRST    |WOMAN  |CHERBOURG      |1           |1        |0        |71.2833  |
|THIRD    |WOMAN  |SOUTHAMPTON    |1           |0        |0        |7.925    |
|FIRST    |WOMAN  |SOUTHAMPTON    |1           |1        |0        |53.1     |
|THIRD    |MAN    |SOUTHAMPTON    |0           |0        |0        |8.05     |
|THIRD    |MAN    |QUEENSTOWN     |0           |0        |0        |8.4583   |
|FIRST    |MAN    |SOUTHAMPTON    |0           |0        |0        |51.8625  |
|THIRD    |CHILD  |SOUTHAMPTON    |0           |3        |1        |21.075   |
|THIRD    |WOMAN  |SOUTHAMPTON    |1           |0        |2        |11.1333  |
|SECOND   |CHILD  |CHERBOURG      |1           |1   

In [10]:
OHE = OneHotEncoder(
    input_cols=cat_cols,
    output_cols=cat_cols,
    drop_input_cols=True,
    drop="first",
    handle_unknown="ignore",
)

titanic_df = OHE.fit(titanic_df).transform(titanic_df)
titanic_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CLASS_SECOND"  |"CLASS_THIRD"  |"WHO_MAN"  |"WHO_WOMAN"  |"EMBARK_TOWN_QUEENSTOWN"  |"EMBARK_TOWN_SOUTHAMPTON"  |"SURVIVED"  |"SIBSP"  |"PARCH"  |"FARE"   |
--------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.0             |1.0            |1.0        |0.0          |0.0                       |1.0                        |0           |1        |0        |7.25     |
|0.0             |0.0            |0.0        |1.0          |0.0                       |0.0                        |1           |1        |0        |71.2833  |
|0.0             |1.0            |0.0        |1.0          |0.0                       |1.0                        |1           |0        |0        |7.925    |
|0.0             |0.0            |0.0        |

In [11]:
train_df, test_df = titanic_df.random_split(weights=[0.8, 0.2], seed=8)

In [12]:
parameters = {
    "n_estimators": [100, 200, 300, 400, 500],
    "learning_rate": [0.1, 0.2, 0.3, 0.4, 0.5],
    "max_depth": list(range(3, 6, 1)),
    "min_child_weight": list(range(1, 6, 1)),
}

In [13]:
parameters

{'n_estimators': [100, 200, 300, 400, 500],
 'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
 'max_depth': [3, 4, 5],
 'min_child_weight': [1, 2, 3, 4, 5]}

In [14]:
session.sql(
    f"ALTER WAREHOUSE {session.get_current_warehouse()[1:-1]} SET WAREHOUSE_SIZE=LARGE;"
).collect()

[Row(status='Statement executed successfully.')]

Data scientists may not have the ability to change the warehouse size.  They will usually have access to a larger warehouse and can easily switch as well using session.use_warehouse('bigger_warehouse')

In [15]:
grid_search = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid=parameters,
    n_jobs=-1,
    scoring="accuracy",
    input_cols=train_df.drop("SURVIVED").columns,
    label_cols="SURVIVED",
    output_cols="PRED_SURVIVED",
)

# Train
grid_search.fit(train_df)

The version of package 'snowflake-snowpark-python' in the local environment is 1.13.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python<2'. Your UDF might not work when the package version is different between the server and your local environment.
Package 'fastparquet' is not installed in the local environment. Your UDF might not work when the package is installed on the server but not on your local environment.
The version of package 'pyarrow' in the local environment is 15.0.1, which does not fit the criteria for the requirement 'pyarrow<14'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'cachetools' in the local environment is 5.3.3, which does not fit the criteria for the requirement 'cachetools<6'. Your UDF might not work when the package version is different between the server and your local environment.


In [16]:
session.sql(
    f"ALTER WAREHOUSE {session.get_current_warehouse()[1:-1]} SET WAREHOUSE_SIZE=XSMALL;"
).collect()

[Row(status='Statement executed successfully.')]

In [17]:
result = grid_search.predict(test_df)

In [18]:
accuracy = accuracy_score(
    df=result, y_true_col_names="SURVIVED", y_pred_col_names="PRED_SURVIVED"
)

print(f"Accuracy: {accuracy}")

Accuracy: 0.819149


In [19]:
# Print each combination of hyperparameters with their accuracy
results = grid_search.to_sklearn().cv_results_
data = {"accuracy": results["mean_test_score"]}
for i, param in enumerate(results["params"]):
    for key, value in param.items():
        if key not in data:
            data[key] = [None] * len(results["params"])
        data[key][i] = value

# Create DataFrame
hp_df = pd.DataFrame(data).sort_values(by="accuracy", ascending=False)
hp_df.head()

,accuracy,learning_rate,max_depth,min_child_weight,n_estimators
94,0.822148,0.2,3,4,500
311,0.820760,0.5,3,3,200
216,0.820760,0.3,5,4,200
114,0.820750,0.2,4,3,500
299,0.820740,0.4,5,5,500


# Model Registry


In [20]:
optimal_model = grid_search.to_sklearn().best_estimator_

In [21]:
# create function to add one to our model versions if it already exists


def check_and_update(df, model_name):
    if df.empty:
        return "V_1"
    elif df[df["name"] == model_name].empty:
        return "V_1"
    else:
        # Increment model_version if df is not a pandas Series
        lst = sorted(ast.literal_eval(df["versions"][0]))
        last_value = lst[-1]
        prefix, num = last_value.rsplit("_", 1)
        new_last_value = f"{prefix}_{int(num)+1}"
        lst[-1] = new_last_value
        return new_last_value

In [22]:
# Get sample input data to pass into the registry logging function
X = train_df.drop("SURVIVED").limit(100)

# Create a registry and log the model
# You can specify a different DB and Schema if you'd like
# otherwise it uses the session context
reg = Registry(session=session)

reg_df = reg.show_models()

# Define model name and version (use uppercase for name)
model_name = "TITANIC"

model_version = check_and_update(reg_df, model_name)

titanic_model = reg.log_model(
    model_name=model_name,
    version_name=model_version,
    model=optimal_model,
    sample_input_data=X,
)

# Add evaluation metric
titanic_model.set_metric(
    metric_name="accuracy",
    value=hp_df["accuracy"][0],
)

In [23]:
reg.show_models()

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-03-01 07:50:43.635000-08:00,TITANIC,SNOWPARK,TITANIC,None,SYSADMIN,V_3,"[""V_1"",""V_2"",""V_3"",""V_4""]"


In [24]:
hyperparameters = {
    k: v for k, v in optimal_model.get_params().items() if v and k != "missing"
}
titanic_model.set_metric(metric_name="hyperparameters", value=hyperparameters)

In [25]:
pd.options.display.max_colwidth = 500
reg.get_model(model_name).show_versions()

,created_on,name,comment,database_name,schema_name,module_name,is_default_version,functions,metadata,user_data
0,2024-03-01 07:50:43.665000-08:00,V_1,None,SNOWPARK,TITANIC,TITANIC,false,"[""PREDICT_PROBA"",""PREDICT"",""APPLY""]","{""metrics"": {""accuracy"": 0.8093617021276595, ""hyperparameters"": {""objective"": ""binary:logistic"", ""learning_rate"": 0.2, ""max_depth"": 3, ""min_child_weight"": 4, ""n_estimators"": 500, ""n_jobs"": 3}}, ""snowpark_ml_schema_version"": ""2024-01-01""}","{""snowpark_ml_data"":{""functions"":[{""name"":""APPLY"",""signature"":{""inputs"":[{""name"":""CLASS_SECOND"",""type"":""DOUBLE""},{""name"":""CLASS_THIRD"",""type"":""DOUBLE""},{""name"":""WHO_MAN"",""type"":""DOUBLE""},{""name"":""WHO_WOMAN"",""type"":""DOUBLE""},{""name"":""EMBARK_TOWN_QUEENSTOWN"",""type"":""DOUBLE""},{""name"":""EMBARK_TOWN_SOUTHAMPTON"",""type"":""DOUBLE""},{""name"":""SIBSP"",""type"":""INT8""},{""name"":""PARCH"",""type"":""INT8""},{""name"":""FARE"",""type"":""DOUBLE""}],""outputs"":[{""name"":""output_feature_0"",""type"":""FLOAT""},{""name"":""output_featur..."
1,2024-03-01 08:11:43.919000-08:00,V_2,None,SNOWPARK,TITANIC,TITANIC,false,"[""PREDICT_PROBA"",""PREDICT""]","{""metrics"": {""accuracy"": 0.887931, ""Precision"": 0.8961038961038961, ""Recall"": 0.7931034482758621, ""F1 Score"": 0.8414634146341463, ""Confusion Matrix"": [[411.0, 24.0], [54.0, 207.0]]}, ""snowpark_ml_schema_version"": ""2024-01-01""}","{""snowpark_ml_data"":{""functions"":[{""name"":""PREDICT"",""signature"":{""inputs"":[{""name"":""PCLASS"",""type"":""INT8""},{""name"":""SIBSP"",""type"":""INT8""},{""name"":""PARCH"",""type"":""INT8""},{""name"":""FARE"",""type"":""DOUBLE""},{""name"":""ALONE"",""type"":""BOOL""},{""name"":""SEX"",""type"":""STRING""},{""name"":""CLASS"",""type"":""STRING""},{""name"":""WHO"",""type"":""STRING""},{""name"":""EMBARK_TOWN"",""type"":""STRING""}],""outputs"":[{""name"":""SEX_MALE"",""type"":""DOUBLE""},{""name"":""CLASS_SECOND"",""type"":""DOUBLE""},{""name"":""CLASS_THIRD"",""type"":""DOUBLE""},{""n..."
2,2024-03-04 13:46:48.044000-08:00,V_3,None,SNOWPARK,TITANIC,TITANIC,true,"[""PREDICT_PROBA"",""PREDICT"",""APPLY""]","{""metrics"": {""accuracy"": 0.8014285714285714, ""hyperparameters"": {""objective"": ""binary:logistic"", ""learning_rate"": 0.5, ""max_depth"": 4, ""min_child_weight"": 3, ""n_estimators"": 100, ""n_jobs"": 3}}, ""snowpark_ml_schema_version"": ""2024-01-01""}","{""snowpark_ml_data"":{""functions"":[{""name"":""APPLY"",""signature"":{""inputs"":[{""name"":""CLASS_SECOND"",""type"":""DOUBLE""},{""name"":""CLASS_THIRD"",""type"":""DOUBLE""},{""name"":""WHO_MAN"",""type"":""DOUBLE""},{""name"":""WHO_WOMAN"",""type"":""DOUBLE""},{""name"":""EMBARK_TOWN_QUEENSTOWN"",""type"":""DOUBLE""},{""name"":""EMBARK_TOWN_SOUTHAMPTON"",""type"":""DOUBLE""},{""name"":""SIBSP"",""type"":""INT8""},{""name"":""PARCH"",""type"":""INT8""},{""name"":""FARE"",""type"":""DOUBLE""}],""outputs"":[{""name"":""output_feature_0"",""type"":""FLOAT""},{""name"":""output_featur..."
3,2024-03-14 06:37:27.315000-07:00,V_4,None,SNOWPARK,TITANIC,TITANIC,false,"[""PREDICT_PROBA"",""PREDICT"",""APPLY""]","{""metrics"": {""accuracy"": 0.8093617021276595, ""hyperparameters"": {""objective"": ""binary:logistic"", ""learning_rate"": 0.2, ""max_depth"": 3, ""min_child_weight"": 4, ""n_estimators"": 500, ""n_jobs"": 3}}, ""snowpark_ml_schema_version"": ""2024-01-01""}","{""snowpark_ml_data"":{""functions"":[{""name"":""APPLY"",""signature"":{""inputs"":[{""name"":""CLASS_SECOND"",""type"":""DOUBLE""},{""name"":""CLASS_THIRD"",""type"":""DOUBLE""},{""name"":""WHO_MAN"",""type"":""DOUBLE""},{""name"":""WHO_WOMAN"",""type"":""DOUBLE""},{""name"":""EMBARK_TOWN_QUEENSTOWN"",""type"":""DOUBLE""},{""name"":""EMBARK_TOWN_SOUTHAMPTON"",""type"":""DOUBLE""},{""name"":""SIBSP"",""type"":""INT8""},{""name"":""PARCH"",""type"":""INT8""},{""name"":""FARE"",""type"":""DOUBLE""}],""outputs"":[{""name"":""output_feature_0"",""type"":""FLOAT""},{""name

If you have multiple versions of the model, we want the UDF to be deployed as the version with the highest accuracy


In [26]:
reg_df = reg.get_model(model_name).show_versions()
reg_df["accuracy"] = reg_df["metadata"].apply(
    lambda x: json.loads(x)["metrics"]["accuracy"]
)
best_model = reg_df.sort_values(by="accuracy", ascending=False)

In [27]:
deployed_version = best_model["name"].iloc[0]
deployed_version

'V_2'

Set the default version to the deployed version (best model)

In [28]:
m = reg.get_model(model_name)
m.default = deployed_version
mv = m.default
mv.version_name

'V_2'

In [29]:
remote_prediction = mv.run(test_df, function_name="predict_proba")
remote_prediction.show()

ValueError: (2112) 
Data Validation Error when validating your Snowpark DataFrame.
If using the normalized names from model signatures, there are the following errors:
[ValueError('Data Validation Error: feature PCLASS does not exist in data.'), ValueError('Data Validation Error: feature ALONE does not exist in data.'), ValueError('Data Validation Error: feature SEX does not exist in data.'), ValueError('Data Validation Error: feature CLASS does not exist in data.'), ValueError('Data Validation Error: feature WHO does not exist in data.'), ValueError('Data Validation Error: feature EMBARK_TOWN does not exist in data.')]

If using the inferred names from model signatures, there are the following errors:
[ValueError('Data Validation Error: feature PCLASS does not exist in data.'), ValueError('Data Validation Error: feature ALONE does not exist in data.'), ValueError('Data Validation Error: feature SEX does not exist in data.'), ValueError('Data Validation Error: feature CLASS does not ex

In [ ]:
# To test in SQL write test data back to a table

test_df.write.mode("overwrite").save_as_table("TEST_DATA")

## Add images to stage for Streamlit App


In [ ]:
session.file.put("../streamlit_images/*", "@ML_DATA")

# Calling model from a new notebook

In [ ]:
# Point to the registry

reg = Registry(session=session)

# Get the default version of your model (Model with best accuracy in our case)

mv = reg.get_model("titanic").default

remote_prediction = mv.run(test_df, function_name="predict_proba")
remote_prediction.drop('"output_feature_0"').with_column_renamed(
    '"output_feature_1"', "pred_survived"
).show()

## To delete your model and all of it's versions

In [30]:
#reg.delete_model("TITANIC")